In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 最终修复版：大规模HMDA预处理
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
import gc
from datetime import datetime

warnings.filterwarnings('ignore')
np.random.seed(42)

def create_large_hmda_dataset_fixed(file_path, target_samples=100000):
    """
    创建大规模HMDA数据集 - 修复版本
    """
    print(f"🎯 目标创建 {target_samples:,} 样本的平衡数据集")
    target_per_class = target_samples // 2

    try:
        # 步骤1: 加载大规模数据
        print(f"\n=== 步骤1: 大规模数据加载 ===")

        # 读取更多数据以确保有足够的拒绝记录
        read_size = 500000  # 读取50万行
        print(f"📖 读取 {read_size:,} 行数据...")

        # 定义需要的列
        required_cols = [
            'loan_amount', 'income', 'property_value', 'interest_rate',
            'loan_type', 'loan_purpose', 'applicant_age', 'state_code', 'county_code',
            'derived_race', 'derived_ethnicity', 'derived_sex', 'action_taken'
        ]

        # 读取数据
        df = pd.read_csv(file_path, nrows=read_size, low_memory=False)

        # 选择可用的列
        available_cols = [col for col in required_cols if col in df.columns]
        df = df[available_cols].copy()

        print(f"✅ 数据加载完成: {df.shape}")
        print(f"使用列: {available_cols}")

        # 步骤2: 温和数据清理
        print(f"\n=== 步骤2: 温和数据清理 ===")

        # 筛选批准/拒绝
        df['action_taken'] = pd.to_numeric(df['action_taken'], errors='coerce')
        df_clean = df[df['action_taken'].isin([1, 3])].copy().reset_index(drop=True)

        print(f"筛选批准/拒绝: {len(df)} → {len(df_clean)}")

        # 检查分布
        approved_count = (df_clean['action_taken'] == 1).sum()
        rejected_count = (df_clean['action_taken'] == 3).sum()
        print(f"批准: {approved_count:,}, 拒绝: {rejected_count:,}")

        # 只做最基本的清理
        if 'loan_amount' in df_clean.columns:
            df_clean['loan_amount'] = pd.to_numeric(df_clean['loan_amount'], errors='coerce')
            df_clean = df_clean.dropna(subset=['loan_amount']).reset_index(drop=True)
            df_clean = df_clean[df_clean['loan_amount'] > 1000].reset_index(drop=True)  # 只移除明显错误的

        # 重新检查分布
        approved_after = (df_clean['action_taken'] == 1).sum()
        rejected_after = (df_clean['action_taken'] == 3).sum()
        print(f"清理后 - 批准: {approved_after:,}, 拒绝: {rejected_after:,}")

        # 步骤3: 简化特征工程
        print(f"\n=== 步骤3: 简化特征工程 ===")

        # 保存敏感属性
        sensitive_cols = ['derived_race', 'derived_ethnicity', 'derived_sex']
        available_sensitive = [col for col in sensitive_cols if col in df_clean.columns]
        sensitive_data = df_clean[available_sensitive].copy()
        print(f"敏感属性: {available_sensitive}")

        # 处理数值特征
        numeric_cols = ['loan_amount', 'income', 'property_value', 'interest_rate']
        available_numeric = [col for col in numeric_cols if col in df_clean.columns]

        feature_dfs = []
        feature_names = []

        if available_numeric:
            # 处理数值特征
            numeric_df = df_clean[available_numeric].copy()

            for col in available_numeric:
                numeric_df[col] = pd.to_numeric(numeric_df[col], errors='coerce')
                median_val = numeric_df[col].median()
                numeric_df[col] = numeric_df[col].fillna(median_val)

                # 移除极端异常值
                Q1 = numeric_df[col].quantile(0.01)
                Q99 = numeric_df[col].quantile(0.99)
                numeric_df[col] = numeric_df[col].clip(Q1, Q99)

            # 标准化
            scaler = StandardScaler()
            numeric_df[available_numeric] = scaler.fit_transform(numeric_df[available_numeric])

            feature_dfs.append(numeric_df)
            feature_names.extend(available_numeric)
            print(f"✓ 数值特征: {len(available_numeric)} 个")

        # 处理分类特征
        categorical_cols = ['loan_type', 'loan_purpose', 'applicant_age', 'state_code']
        available_categorical = [col for col in categorical_cols if col in df_clean.columns]

        for col in available_categorical:
            try:
                # 填充缺失值
                df_col = df_clean[col].fillna('Unknown')

                # 限制类别数量
                if df_col.nunique() > 20:
                    top_values = df_col.value_counts().head(15).index
                    df_col = df_col.apply(lambda x: x if x in top_values else 'Other')

                # 独热编码
                dummies = pd.get_dummies(df_col, prefix=col, drop_first=True)
                feature_dfs.append(dummies)
                feature_names.extend(dummies.columns.tolist())
                print(f"  {col}: {len(dummies.columns)} 个特征")

            except Exception as e:
                print(f"  {col}: 编码失败 - {e}")

        # 合并特征
        if not feature_dfs:
            print("❌ 没有可用特征")
            return None

        features_df = pd.concat(feature_dfs, axis=1)
        print(f"✓ 总特征数: {len(feature_names)}")

        # 步骤4: 简单平衡采样
        print(f"\n=== 步骤4: 简单平衡采样 ===")

        # 创建目标变量
        target = (df_clean['action_taken'] == 1).astype(int)

        # 分离正负样本
        approved_mask = target == 1
        rejected_mask = target == 0

        approved_features = features_df[approved_mask].reset_index(drop=True)
        approved_sensitive = sensitive_data[approved_mask].reset_index(drop=True)

        rejected_features = features_df[rejected_mask].reset_index(drop=True)
        rejected_sensitive = sensitive_data[rejected_mask].reset_index(drop=True)

        print(f"分离后 - 批准: {len(approved_features):,}, 拒绝: {len(rejected_features):,}")

        # 确定采样数量
        actual_per_class = min(target_per_class, len(approved_features), len(rejected_features))
        print(f"每类采样数量: {actual_per_class:,}")

        # 简单随机采样
        approved_idx = np.random.choice(len(approved_features), actual_per_class, replace=False)
        rejected_idx = np.random.choice(len(rejected_features), actual_per_class, replace=False)

        # 采样数据
        approved_sampled_features = approved_features.iloc[approved_idx].reset_index(drop=True)
        approved_sampled_sensitive = approved_sensitive.iloc[approved_idx].reset_index(drop=True)

        rejected_sampled_features = rejected_features.iloc[rejected_idx].reset_index(drop=True)
        rejected_sampled_sensitive = rejected_sensitive.iloc[rejected_idx].reset_index(drop=True)

        # 合并数据
        X = pd.concat([approved_sampled_features, rejected_sampled_features], ignore_index=True)
        y = pd.concat([
            pd.Series([1] * actual_per_class),
            pd.Series([0] * actual_per_class)
        ], ignore_index=True)

        sensitive_final = pd.concat([
            approved_sampled_sensitive,
            rejected_sampled_sensitive
        ], ignore_index=True)

        # 打乱数据
        shuffle_idx = np.random.permutation(len(X))
        X = X.iloc[shuffle_idx].reset_index(drop=True)
        y = y.iloc[shuffle_idx].reset_index(drop=True)
        sensitive_final = sensitive_final.iloc[shuffle_idx].reset_index(drop=True)

        # 最终结果
        total_samples = len(X)
        approval_rate = y.mean()

        print(f"\n✅ 大规模数据集创建成功!")
        print(f"📊 最终数据集: {total_samples:,} 条记录")
        print(f"📊 特征数量: {len(feature_names)} 个")
        print(f"📊 批准率: {approval_rate:.1%}")

        # 代理变量分析
        proxy_count = sum(1 for col in feature_names if any(
            keyword in col.lower() for keyword in ['state', 'county', 'age', 'loan', 'type']
        ))
        print(f"📊 代理变量: ~{proxy_count} 个")

        # 按敏感属性分析
        print(f"\n📈 按敏感属性的批准率:")
        for col in available_sensitive:
            if col in sensitive_final.columns:
                print(f"\n{col}:")
                for group in sensitive_final[col].value_counts().head(3).index:
                    mask = sensitive_final[col] == group
                    if mask.sum() > 100:
                        rate = y[mask].mean()
                        count = mask.sum()
                        print(f"  {group}: {rate:.3f} ({count:,} 样本)")

        # 保存数据
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        try:
            X.to_csv(f'hmda_large_final_{timestamp}.csv', index=False)
            labels_df = pd.DataFrame({'target': y})
            for col in sensitive_final.columns:
                labels_df[col] = sensitive_final[col]
            labels_df.to_csv(f'hmda_labels_final_{timestamp}.csv', index=False)
            print(f"\n✅ 数据已保存: hmda_large_final_{timestamp}.csv")
        except Exception as e:
            print(f"⚠️ 保存失败: {e}")

        return {
            'X': X,
            'y': y,
            'sensitive_data': sensitive_final,
            'feature_names': feature_names,
            'stats': {
                'total_samples': total_samples,
                'n_features': len(feature_names),
                'approval_rate': approval_rate,
                'proxy_variables': proxy_count
            }
        }

    except Exception as e:
        print(f"❌ 处理失败: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def quick_hmda_summary(result):
    """
    快速数据集摘要
    """
    if result is None:
        print("❌ 没有可用数据")
        return

    X, y, sensitive_data = result['X'], result['y'], result['sensitive_data']

    print("\n" + "="*60)
    print("🎉 大规模HMDA数据集创建完成")
    print("="*60)
    print(f"📊 训练特征: {X.shape}")
    print(f"📊 目标变量: {y.shape}")
    print(f"📊 敏感属性: {sensitive_data.shape}")
    print(f"📊 批准率: {y.mean():.1%}")

    # 数据质量检查
    print(f"\n🔍 数据质量:")
    print(f"  无缺失值: {X.isnull().sum().sum() == 0}")
    print(f"  特征范围: {X.std().mean():.3f}")
    print(f"  标签平衡: {'✓' if abs(y.mean() - 0.5) < 0.01 else '✗'}")

    print(f"\n🚀 数据已准备就绪，可以开始神经网络训练!")
    print("="*60)

# 运行函数
if __name__ == "__main__":
    file_path = "/content/drive/MyDrive/2023_public_lar_csv.csv"

    # 创建大规模数据集
    print("开始创建大规模HMDA数据集...")
    result = create_large_hmda_dataset_fixed(file_path, target_samples=100000)

    # 显示摘要
    quick_hmda_summary(result)

    if result:
        print(f"\n🎯 成功创建了100K样本的平衡数据集!")
        print(f"现在可以开始建模了...")

开始创建大规模HMDA数据集...
🎯 目标创建 100,000 样本的平衡数据集

=== 步骤1: 大规模数据加载 ===
📖 读取 500,000 行数据...
✅ 数据加载完成: (500000, 13)
使用列: ['loan_amount', 'income', 'property_value', 'interest_rate', 'loan_type', 'loan_purpose', 'applicant_age', 'state_code', 'county_code', 'derived_race', 'derived_ethnicity', 'derived_sex', 'action_taken']

=== 步骤2: 温和数据清理 ===
筛选批准/拒绝: 500000 → 316734
批准: 227,827, 拒绝: 88,907
清理后 - 批准: 227,827, 拒绝: 88,907

=== 步骤3: 简化特征工程 ===
敏感属性: ['derived_race', 'derived_ethnicity', 'derived_sex']
✓ 数值特征: 4 个
  loan_type: 3 个特征
  loan_purpose: 5 个特征
  applicant_age: 7 个特征
  state_code: 15 个特征
✓ 总特征数: 34

=== 步骤4: 简单平衡采样 ===
分离后 - 批准: 227,827, 拒绝: 88,907
每类采样数量: 50,000

✅ 大规模数据集创建成功!
📊 最终数据集: 100,000 条记录
📊 特征数量: 34 个
📊 批准率: 50.0%
📊 代理变量: ~31 个

📈 按敏感属性的批准率:

derived_race:
  White: 0.509 (64,945 样本)
  Race Not Available: 0.560 (15,145 样本)
  Black or African American: 0.336 (10,991 样本)

derived_ethnicity:
  Not Hispanic or Latino: 0.500 (72,594 样本)
  Ethnicity Not Available: 0.578 (13,595 样本)
  